In [9]:
import pyam
import pandas as pd
import nomenclature
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [67]:
df = pd.read_excel('../../data/GEO7_FOD_results.xlsx', sheet_name='data')
df = df[['Model', 'Scenario', 'Region', 'Variable', 'Unit', '2010', '2015', '2020', '2025', '2030', '2035', '2040', '2045', '2050']]
df.columns = df.columns.str.lower()
df

,model,scenario,region,variable,unit,2010,2015,2020,2025,2030,2035,2040,2045,2050
0,IMAGE 3.3,LIFE-TP,Africa (UN-R5),Agricultural Demand,million t DM/yr,421.076129,488.757398,552.659523,622.599414,695.450176,815.475719,1374.956355,1877.400082,2097.145465
1,IMAGE 3.3,LIFE-TP,Africa (UN-R5),Agricultural Demand|Energy,million t DM/yr,3.022935,8.107065,9.120188,20.039323,29.254952,72.963684,556.594769,993.819725,1148.080659
2,IMAGE 3.3,LIFE-TP,Africa (UN-R5),Agricultural Demand|Energy|Crops,million t DM/yr,3.022935,8.107065,9.120188,20.039323,29.254952,72.963684,556.594769,993.819725,1148.080659
3,IMAGE 3.3,LIFE-TP,Africa (UN-R5),Agricultural Demand|Energy|Crops|1st generation,million t DM/yr,3.022935,8.107065,9.120188,3.146959,2.949206,3.196734,13.290263,15.811367,13.543890
4,IMAGE 3.3,LIFE-TP,Africa (UN-R5),Agricultural Demand|Energy|Crops|2nd generation,million t DM/yr,0.000000,0.000000,0.000000,16.892364,26.305745,69.766952,543.304492,978.008369,1134.536791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3246,JRC-FASST 1.0.0,REF,World,Concentration|PM2.5,µg/m3,NaN,34.585931,34.006688,35.148099,35.371155,NaN,NaN,32.979457,32.402158
3247,JRC-FASST 1.0.0,TECH-TP,World,Concentration|PM2.5,µg/m3,NaN,34.530652,33.806212,29.448617,29.212242,NaN,NaN,27.588576,30.605810
3248,JRC-FASST 1.0.0,LIFE-TP,World,Population|Exposure to Ambient PM2.5 > 25 μg/m3,%,NaN,59.858591,59.573252,52.696135,49.210350,NaN,NaN,50.025408,50.397561
3249,JRC-FASST 1.0.0,REF,World,Population|Exposure to Ambient PM2.5 > 25 μg/m3,%,NaN,59.941379,59.764277,61.189189,61.763281,NaN,NaN,57.769602,56.295155


In [69]:
def plot_global_scenarios(df, var_dict):
    # Selected variables and transform dataframe
    df_world = df[df['region'] == 'World']
    df_selected = df_world[df_world['variable'].isin(list(var_dict.keys()))]
    variables = df_selected['variable'].unique()
    df_melted = pd.melt(df_selected, id_vars=['model',
                              'scenario',
                              'region',
                              'variable',
                              'unit'
                                ], var_name='year', value_name='value')
    df_melted['year'] = df_melted['year'].astype(int)
    # Loop throguh vars
    for variable in variables:
        # Use Seaborn to create the plot with markers and lines
        data = df_melted[df_melted["variable"] == variable]
        
        # Define the color palette for scenarios, might be useful as a global thing
        scenario_colors = {
            "REF": "black",
            "REF-1p5C": "blue",
            "LIFE-TP": "orange",
            "TECH-TP": "magenta",
        }

        plt.figure(figsize=(10, 6))  # Set figure size
        sns.lineplot(
            data=data,
            x="year",
            y="value",
            hue="scenario",
            palette=scenario_colors,
            linewidth=2.5
        )

        
        # Set y-axis lower limit to 0
        ax = plt.gca()
        ax.set_ylim(min(0,ax.get_ylim()[0]), ax.get_ylim()[1])
        
        # Set labels and title
        plt.xlabel("year", fontsize=12)
        plt.ylabel(data["unit"].iloc[0], fontsize=12)  # Set the y-axis label to the unit
        plt.title(var_dict[variable]['title'])  # Set the title to the variable
        plt.xticks([2010, 2020, 2030, 2040, 2050], 
                rotation=45,
                fontsize=12)
        plt.yticks(fontsize=12)
        
        # Show legend and grid
        plt.legend(title="Legend", bbox_to_anchor=(1.3, 1))
        plt.grid(True)
        # Adjust layout to ensure legend fits within the saved image
        plt.tight_layout()
        # Save the figure
        path = var_dict[variable]['path']
        variable_name = variable.replace("|", "_").replace(' > ', '').replace(" ", "_").replace("/", "")
        plt.savefig(f"../../plots/chpt_20/Global/{path}/{variable_name}.png")
        plt.close()

var_dict = {
    "Land Cover|Cropland": {"title" : "Land Cover Cropland", "path":"Systems/Food"}, 
    "Land Cover|Pasture": {"title" : "Land Cover Pasture", "path":"Systems/Food"},
    "Land Cover|Forest": {"title" : "Forest Area", "path":"Systems/Environment"},
    "Primary Energy": {"title" : "Total Primary Energy", "path":"Systems/Energy"},
    "Terrestrial Biodiversity|MSA":{"title" : "Mean Species Abundance", "path":"Goals/Biodiversity_Loss"},
    'Emissions|Kyoto Gases': {"title" : "Greenhouse Gas Emissions", "path":"Goals/Climate_Change"},
    'Temperature|Global Mean': {"title" : "Global Mean Temperature", "path":"Goals/Climate_Change"},
    'Population|Exposure to Ambient PM2.5 > 25 μg/m3': {"title" : "Population exposed to ambient PM2.5 > 25 μg/m^3", "path":"Goals/Pollution"},
}

plot_global_scenarios(df, var_dict)

# Calculate new variables

In [76]:
# Use pyam functionalities to create new variables
pyam_df = pyam.IamDataFrame(data=df)
# (Primary Energy|Biomass + Primary Energy|Hydro + Primary Energy|Solar + Primary Energy|Wind + Primary Energy|Other) / Primary Energy
# Agricultural Demand|Non-Energy|Crops|Food / (Agricultural Demand|Non-Energy|Crops|Food + Agricultural Demand|Non-Energy|Livestock|Food)
pyam_df.aggregate("Primary Energy|Renewables", 
                       components=['Primary Energy|Biomass', 
                                    'Primary Energy|Hydro', 
                                    'Primary Energy|Solar', 
                                    'Primary Energy|Wind',
                                    'Primary Energy|Other'], 
                        append=True)

pyam_df.aggregate("Agricultural Demand|Non-Energy|Crops & Livestock|Food", 
                       components=['Agricultural Demand|Non-Energy|Crops|Food', 
                                    'Agricultural Demand|Non-Energy|Livestock|Food',
                                    ], 
                        append=True)

pyam_df.divide("Primary Energy|Renewables", 
               "Primary Energy", 
               "Primary Energy|Renewables (Fraction)", 
               ignore_units=True, 
               append=True)

pyam_df.multiply("Primary Energy|Renewables (Fraction)", 
               100, 
               "Primary Energy|Renewables (Share)", 
               ignore_units='%', 
               append=True)


pyam_df.divide("Agricultural Demand|Non-Energy|Crops|Food", 
               "Agricultural Demand|Non-Energy|Crops & Livestock|Food", 
               "Agricultural Demand|Non-Energy|Crops|Food (Fraction)", 
               ignore_units=True, 
               append=True)

pyam_df.multiply("Agricultural Demand|Non-Energy|Crops|Food (Fraction)", 
               100, 
               "Agricultural Demand|Non-Energy|Crops|Food (Share)", 
               ignore_units='%', 
               append=True)

var_dict = {"Primary Energy|Renewables (Share)": {"title" : "Share of renewables in total primary energy", "path":"Systems/Energy"}, 
            "Agricultural Demand|Non-Energy|Crops|Food (Share)": {"title" : "Share of crops in total agricultural demand", "path":"Systems/Food"}}

df_solid = pyam_df.filter(variable=["Primary Energy|Renewables (Share)", 
                                    "Agricultural Demand|Non-Energy|Crops|Food (Share)"],
                          region="World",
                          year=[2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050])

df_solid_timeseries = df_solid.timeseries().reset_index()


plot_global_scenarios(df_solid_timeseries, var_dict)

# Get data from database

In [18]:
# AIM has not submitted, use IMAGE
# (Land Cover|Forest + Land Cover|Other Land) / (Land Cover)
# Population|Electricity Access / Population
# Population relying on solid fuels / Population

df_calc = pyam.read_iiasa(
    'geo7_internal',
    variable=['Population|Relying on Solid Fuels',
              'Land Cover|Forest',
              'Land Cover|Other Land',
              'Land Cover',
              'Population|Electricity Access',
              'Population',
              'Production|Non-Metallic Minerals|Cement',
              'Emissions|CH4|AFOLU',
              'Emissions|CH4|Energy',
              'Emissions|CH4|Industrial Processes',
              'Emissions|CH4|Other',
              'Emissions|CH4|Waste',],
    region='World',
    model='IMAGE 3.3',
)


ixmp4.conf.settings - INFO: Connecting as user 'LaszloVreedenburgh'.
2024-02-22 09:11:27 INFO     Connecting as user 'LaszloVreedenburgh'.
pyam.iiasa - INFO: You are connected to the IXSE_GEO7_INTERNAL scenario explorer hosted by IIASA. If you use this data in any published format, please cite the data as provided in the explorer guidelines: https://data.ece.iiasa.ac.at/geo7-internal/#/about
2024-02-22 09:11:28 INFO     You are connected to the IXSE_GEO7_INTERNAL scenario explorer hosted by IIASA. If you use this data in any published format, please cite the data as provided in the explorer guidelines: https://data.ece.iiasa.ac.at/geo7-internal/#/about
pyam.iiasa - INFO: You are connected as user `LaszloVreedenburgh`
2024-02-22 09:11:28 INFO     You are connected as user `LaszloVreedenburgh`


In [19]:
# Create new variables
df_calc.aggregate("Land Cover|Forest & Other Land", 
                    components=['Land Cover|Forest', 
                                'Land Cover|Other Land',], 
                    append=True)

In [20]:
df_calc.divide("Population|Relying on Solid Fuels", 
               "Population", 
               "Population|Relying on Solid Fuels (Fraction)", 
               ignore_units=True, 
               append=True)
df_calc.multiply("Population|Relying on Solid Fuels (Fraction)", 
               100, 
               "Population|Relying on Solid Fuels (Share)", 
               ignore_units='%', 
               append=True)

df_calc.divide("Population|Electricity Access", 
               "Population", 
               "Population|Electricity Access (Fraction)", 
               ignore_units=True, 
               append=True)
df_calc.multiply("Population|Electricity Access (Fraction)", 
               100, 
               "Population|Electricity Access (Share)", 
               ignore_units='%', 
               append=True)

df_calc.divide("Land Cover|Forest & Other Land", 
               "Land Cover", 
               "Land Cover|Forest & Other Land (Fraction)", 
               ignore_units='%', 
               append=True)
df_calc.multiply("Land Cover|Forest & Other Land (Fraction)", 
               100, 
               "Land Cover|Forest & Other Land (Share)", 
               ignore_units='%', 
               append=True)
df_calc.aggregate("Emissions|CH4", append=True)

In [22]:
compare_dict = {"Land Cover|Forest & Other Land (Share)": "Share of forest and other natural land", 
                "Population|Electricity Access (Share)": "Electricity Access",
                "Population|Relying on Solid Fuels (Share)": "Population relying on solid fuels",
                'Production|Non-Metallic Minerals|Cement': "Production of cement",
                "Emissions|CH4": "Emissions|CH4"}

path_dict = {"Land Cover|Forest & Other Land (Share)": "Goals/Land_Degradation", 
             "Population|Electricity Access (Share)": "Goals/Energy_Access",
             "Population|Relying on Solid Fuels (Share)": "Goals/Energy_Access",
                'Production|Non-Metallic Minerals|Cement': "Systems/Circularity",
                "Emissions|CH4": "Other/Emissions"}

df_calc_filtered = df_calc.filter(scenario=['LIFE-TP', 'REF', 'TECH-TP'],
                         variable=["Land Cover|Forest & Other Land (Share)",
                                   "Population|Electricity Access (Share)",
                                   "Population|Relying on Solid Fuels (Share)",
                                   'Production|Non-Metallic Minerals|Cement',
                                   'Emissions|CH4'],
                         region='World',
                         year=[2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050])
df_calc_data = df_calc_filtered.data
variables = df_calc_data.variable.unique()
for variable in variables:
    print(variable)
    # Use Seaborn to create the plot with markers and lines
    data = df_calc_data[df_calc_data["variable"] == variable]

    # Define the color palette for scenarios
    scenario_colors = {
        "REF": "black",
        "REF-1p5C": "blue",
        "LIFE-TP": "orange",
        "TECH-TP": "magenta",
    }

    plt.figure(figsize=(10, 6))  # Set figure size
    sns.lineplot(
        data=data,
        x="year",
        y="value",
        hue="scenario",
        dashes=True,
        palette=scenario_colors,
    )

    # Set labels and title
    plt.xlabel("Year", fontsize=12)
    plt.ylabel(data["unit"].iloc[0], fontsize=12)  # Set the y-axis label to the unit
    # Set y-axis lower limit to 0
    ax = plt.gca()
    ax.set_ylim(min(0,ax.get_ylim()[0]), ax.get_ylim()[1])
    plt.title(compare_dict[variable])  # Set the title to the variable
    plt.xticks([2010, 2020, 2030, 2040, 2050,], 
               rotation=45,
               fontsize=12)
    plt.yticks(fontsize=12)

    # Show legend and grid
    plt.legend(title="Legend", bbox_to_anchor=(1.15, 1))
    plt.grid(True)
    # Adjust layout to ensure legend fits within the saved image
    plt.tight_layout()
    root_variable = variable.split("|")[0].replace(" ", "_")
    variable_name = variable.replace("|", "_").replace(" ", "_").replace("/", "")
    plt.savefig(f"../../plots/chpt_20/Global/{path_dict[variable]}/{variable_name}.png")
    plt.close()

Emissions|CH4
Land Cover|Forest & Other Land (Share)
Population|Electricity Access (Share)
Population|Relying on Solid Fuels (Share)
Production|Non-Metallic Minerals|Cement


# Hunger and Poverty

In [23]:
df_hunger = pyam.read_iiasa(
    'geo7_internal',
    variable=['Population|Risk of Hunger', 'Population|Extreme Poverty|International Poverty Line'],
    region=['World'],
    model=['AIM/Hub-Global 2.4', 'IMAGE 3.3'],
    scenario=['LIFE-TP', 'REF', 'TECH-TP']
)

pyam.iiasa - INFO: You are connected to the IXSE_GEO7_INTERNAL scenario explorer hosted by IIASA. If you use this data in any published format, please cite the data as provided in the explorer guidelines: https://data.ece.iiasa.ac.at/geo7-internal/#/about
2024-02-22 09:11:49 INFO     You are connected to the IXSE_GEO7_INTERNAL scenario explorer hosted by IIASA. If you use this data in any published format, please cite the data as provided in the explorer guidelines: https://data.ece.iiasa.ac.at/geo7-internal/#/about
pyam.iiasa - INFO: You are connected as user `LaszloVreedenburgh`
2024-02-22 09:11:49 INFO     You are connected as user `LaszloVreedenburgh`


In [24]:
compare_dict = {"Population|Risk of Hunger": "Population at risk of hunger", 
                'Population|Extreme Poverty|International Poverty Line':'Population|Extreme Poverty|International Poverty Line' }

path_dict = {"Population|Risk of Hunger": "Goals/Food_Security",
             'Population|Extreme Poverty|International Poverty Line':'Other/Population'
             }

df_hunger_data = df_hunger.filter(year=[2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050]).data
# Condition to identify rows to be removed
condition_image = (df_hunger_data['model'] == 'IMAGE 3.3') & (df_hunger_data['scenario'] == 'REF')

# Remove multiple scenarios for aim
aim_scen_to_remove = ['TECH-TP', 'LIFE-TP']
condition_aim = (df_hunger_data['model'] == 'AIM/Hub-Global 2.4') & (df_hunger_data['scenario'].isin(aim_scen_to_remove))

# Remove rows based on the condition
df_hunger_data = df_hunger_data[~condition_image]
df_hunger_data = df_hunger_data[~condition_aim]

df_hunger_data = df_hunger_data.reset_index(drop=True)

variables = df_hunger_data.variable.unique()

for variable in variables:
    # Use Seaborn to create the plot with markers and lines
    data = df_hunger_data[df_hunger_data["variable"] == variable]

    # Define the color palette for scenarios
    scenario_colors = {
        "REF": "black",
        "REF-1p5C": "blue",
        "LIFE-TP": "orange",
        "TECH-TP": "magenta",
    }

    plt.figure(figsize=(10, 6))  # Set figure size
    sns.lineplot(
        data=data,
        x="year",
        y="value",
        hue="scenario",
        dashes=True,
        palette=scenario_colors,
    )

    # Set labels and title
    plt.xlabel("Year", fontsize=12)
    plt.ylabel(data["unit"].iloc[0], fontsize=12)  # Set the y-axis label to the unit
    # Set y-axis lower limit to 0
    ax = plt.gca()
    ax.set_ylim(min(0,ax.get_ylim()[0]), ax.get_ylim()[1])
    plt.title(compare_dict[variable])  # Set the title to the variable
    plt.xticks([2010, 2020, 2030, 2040, 2050], 
               rotation=45,
               fontsize=12)
    plt.yticks(fontsize=12)

    # Show legend and grid
    plt.legend(title="Legend", bbox_to_anchor=(1.3, 1))
    plt.grid(True)
    # Adjust layout to ensure legend fits within the saved image
    plt.tight_layout()
    root_variable = variable.split("|")[0].replace(" ", "_")
    variable_name = variable.replace("|", "_").replace(" ", "_").replace("/", "")
    plt.savefig(f"../../plots/chpt_20/Global/{path_dict[variable]}/{variable_name}.png")
    plt.close()

C:\Users\vreedenbul\AppData\Local\Temp\ipykernel_25032\1090396437.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hunger_data = df_hunger_data[~condition_aim]
